In [1]:
!nvidia-smi

/usr/bin/zsh: /home/dimask/miniconda3/lib/libtinfo.so.6: no version information available (required by /usr/bin/zsh)
Fri Mar 17 01:35:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.05    Driver Version: 528.24       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:04:00.0  On |                  N/A |
|  0%   51C    P8    42W / 420W |    736MiB / 24576MiB |     12%      Default |
|                               |                      |                  N/A |
+-------------------------------+------------------

<h3> 3. Edit Config For Transfer Learning </h3>

In [2]:
import os
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

2023-03-17 01:35:25.718985: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/dimask/miniconda3/lib/:/home/dimask/miniconda3/envs/tfod3/lib/
2023-03-17 01:35:25.721633: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/dimask/miniconda3/lib/:/home/dimask/miniconda3/envs/tfod3/lib/
2023-03-17 01:35:25.721660: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


<h4>3.a Read config util form pre-trained models that you have download </h4>

In [3]:

PIPELINE_FILE_NAME = 'pipeline.config'
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
LABEL_MAP_NAME = 'label_map.pbtxt'

paths = {'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
         'TFRECORD_PATH': os.path.join('Tensorflow','workspace','coco','TFrecord'),
         'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
         'COCO_SCRIPT_PATH' : os.path.join('Tensorflow','workspace','coco','Script'),
         }

config = config_util.get_configs_from_pipeline_file(os.path.join(paths['PRETRAINED_MODEL_PATH'],PRETRAINED_MODEL_NAME ,PIPELINE_FILE_NAME))

In [4]:
config

{'model': ssd {
   num_classes: 90
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matc

<h4>3.b read and edit config file to define the path of Label_Map, Checkpoint, TFrecord File COCO_train.record and coco_val.record </h4>

In [5]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(os.path.join(paths['PRETRAINED_MODEL_PATH'],PRETRAINED_MODEL_NAME ,PIPELINE_FILE_NAME), "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [6]:
pipeline_config.model.ssd.num_classes = 4 #change number with num of class 
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'],PRETRAINED_MODEL_NAME,'checkpoint','ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= os.path.join(paths['ANNOTATION_PATH'],LABEL_MAP_NAME)
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['TFRECORD_PATH'],'coco_train.record')]
pipeline_config.eval_input_reader[0].label_map_path = os.path.join(paths['ANNOTATION_PATH'],LABEL_MAP_NAME)
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['TFRECORD_PATH'],'coco_val.record')]

In [7]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(os.path.join(paths['COCO_SCRIPT_PATH'],PIPELINE_FILE_NAME), "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)  

In [8]:
config = config_util.get_configs_from_pipeline_file(os.path.join(paths['COCO_SCRIPT_PATH'],PIPELINE_FILE_NAME))

In [9]:
config

{'model': ssd {
   num_classes: 4
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   match

<h3>3.c Training Step </h3>

In [10]:
TRAINING_SCRIPT = './Tensorflow/models/research/object_detection/model_main_tf2.py'


In [11]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=2000".format(TRAINING_SCRIPT, './Tensorflow/workspace/mymodel',os.path.join(paths['COCO_SCRIPT_PATH'],PIPELINE_FILE_NAME))

In [12]:
!{command}

/usr/bin/zsh: /home/dimask/miniconda3/lib/libtinfo.so.6: no version information available (required by /usr/bin/zsh)
2023-03-17 01:36:36.133123: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/dimask/miniconda3/lib/:/home/dimask/miniconda3/envs/tfod3/lib/
2023-03-17 01:36:36.135781: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/dimask/miniconda3/lib/:/home/dimask/miniconda3/envs/tfod3/lib/
2023-03-17 01:36:36.135862: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the miss